In [ ]:
import sys
sys.path.append("../")

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import json

In [ ]:
import Misc.env_vars as ev

In [ ]:
from Models.models import dq_col_sum, dq_agg_sum, dq_agg_view
from Logic.base_queries import run_stmt
import Misc.env_vars as ev

In [ ]:
from Logic import base_queries as bq

In [ ]:
from sqlalchemy import Select, func, or_, desc

## Top Cards 

### Total Orders with Delta

In [ ]:
dt_rng = bq.get_date_range()

max_date = dt_rng[1]

In [ ]:
max_date

In [ ]:
prev_dt = max_date - timedelta(days=1)

In [ ]:
top_cards = (
    Select(
        dq_agg_sum.c.ord_date.label("Order_Date"),
        func.sum(dq_agg_sum.c.total_orders).label("Total_Orders"),
        func.sum(dq_agg_sum.c.total_canceled_orders).label("Cancelled_Orders")
    ).where(or_(dq_agg_sum.c.ord_date == max_date, dq_agg_sum.c.ord_date == prev_dt))
    .group_by(dq_agg_sum.c.ord_date)
    .order_by(desc(dq_agg_sum.c.ord_date))
)

In [ ]:
print(top_cards)

In [ ]:
res = run_stmt(top_cards)

In [ ]:
df_temp = pd.DataFrame(res)

In [ ]:
# df_temp.loc[:,["Order_Date"]] = df_temp["Order_Date"].astype()
df_temp.loc[:,"Total_Orders"] = df_temp["Total_Orders"].astype(int)
df_temp.loc[:,"Cancelled_Orders"] = df_temp["Cancelled_Orders"].astype(int)

In [ ]:
df_temp["Cancel_percentage"] = df_temp["Cancelled_Orders"] / df_temp["Total_Orders"]

In [ ]:
df_temp["Completed_percentage"] = (df_temp["Total_Orders"] - df_temp["Cancelled_Orders"])/ df_temp["Total_Orders"]

In [ ]:
df_temp

In [ ]:
def calc_metrices(df: pd.DataFrame, col_name: str):
    old_val = np.round(df[col_name][1],4)
    new_val = np.round(df[col_name][0],4)
    diff = new_val - old_val
    per_diff = np.round((diff/old_val)*100,4)
    return new_val, diff, per_diff

In [ ]:
tt, td, tp = calc_metrices(df_temp, "Total_Orders")

In [ ]:
total_orders = {
    "title":"Total Orders",
    "count": str(tt),
    "increased" : False if td < 0 else True,
    "variancePercentage" : str(tp),
    "varianceText": "vs Yesterday"
}

In [ ]:
tc, cd, cp = calc_metrices(df_temp, "Cancelled_Orders")

In [ ]:
total_cancellation = {
    "title":"Cancelled Orders",
    "count": str(tc),
    "increased" : False if cd < 0 else True,
    "variancePercentage" : str(cp),
    "varianceText": "vs Yesterday"
}

In [ ]:
cct, ccd, ccp = calc_metrices(df_temp, "Cancel_percentage")

In [ ]:
cct

In [ ]:
cancel_percentage = {
    "title":"Order Cancellation %",
    "count": str(cct),
    "increased" : False if ccd < 0 else True,
    "variancePercentage" : str(ccp),
    "varianceText": "vs Yesterday"
}

In [ ]:
cot, cod, cop = calc_metrices(df_temp, "Completed_percentage")

In [ ]:
compl_percentage = {
    "title":"Order Completion %",
    "count": str(cot),
    "increased" : False if cod < 0 else True,
    "variancePercentage" : str(cop),
    "varianceText": "vs Yesterday"
}

In [ ]:
final_list = [total_orders, total_cancellation, cancel_percentage,compl_percentage]

In [ ]:
final_list

## Missing Columns Completed

In [ ]:
col_sum_comp = (
    Select(
        dq_agg_view.c.ord_date, 
        func.sum(dq_agg_view.c.total_orders).label("total_orders"),
        func.sum(dq_agg_view.c.null_fulfilment_id).label("null_fulfilment_id"),
        func.sum(dq_agg_view.c.null_net_tran_id).label("null_net_tran_id"),
        func.sum(dq_agg_view.c.null_qty.label("null_qty")),
        func.sum(dq_agg_view.c.null_itm_fulfilment_id).label("null_itm_fulfilment_id"),
        func.sum(dq_agg_view.c.null_del_pc).label("null_del_pc"),
        func.sum(dq_agg_view.c.null_created_date_time).label("null_created_date_time"),
        func.sum(dq_agg_view.c.null_domain).label("null_domain"),
        func.sum(dq_agg_view.c.null_del_cty).label("null_del_cty"),
        func.sum(dq_agg_view.c.null_ord_stats).label("null_ord_stats"),
        func.sum(dq_agg_view.c.null_fulfil_status).label("null_fulfil_status"),
        func.sum(dq_agg_view.c.null_itm_cat).label("null_itm_cat"),
        func.sum(dq_agg_view.c.null_cat_cons).label("null_cat_cons"),
        func.sum(dq_agg_view.c.null_sell_pincode).label("null_sell_pincode"),
        func.sum(dq_agg_view.c.null_prov_id).label("null_prov_id"),
        func.sum(dq_agg_view.c.null_itm_id).label("null_itm_id"),
        func.sum(dq_agg_view.c.null_sell_np).label("null_sell_np"),
        func.sum(dq_agg_view.c.null_net_ord_id).label("null_net_ord_id"),
        func.sum(dq_agg_view.c.null_sell_cty).label("null_sell_cty"),
    ).where(or_(dq_agg_view.c.ord_date==max_date, dq_agg_view.c.ord_date==prev_dt))
    .group_by(dq_agg_view.c.ord_date)
    .order_by(desc(dq_agg_view.c.ord_date))
)

In [ ]:
col_sum_canc = (
    Select(
        dq_agg_view.c.ord_date, 
        func.sum(dq_agg_view.c.total_canceled_orders).label("total_canceled_orders"),
        func.sum(dq_agg_view.c.null_cans_code).label("null_cans_code"),
        func.sum(dq_agg_view.c.null_cans_dt_time).label("null_cans_dt_time")
    ).where(or_(dq_agg_view.c.ord_date==max_date, dq_agg_view.c.ord_date==prev_dt))
    .group_by(dq_agg_view.c.ord_date)
    .order_by(desc(dq_agg_view.c.ord_date))
)

In [ ]:
print(col_sum_comp)

In [ ]:
res = run_stmt(col_sum_comp)

In [ ]:
res_canc = run_stmt(col_sum_canc)

In [ ]:
tmp_canc = pd.DataFrame(res_canc)

In [ ]:
tmp_canc

In [ ]:
tmp_df = pd.DataFrame(res)

In [ ]:
tmp_df

In [ ]:
# per_df = pd.DataFrame()

In [ ]:
# per_df["ord_date"] = tmp_df["ord_date"]

In [ ]:
# per_df

### Calculate Percentage and Difference 

In [ ]:
curr_total = int(tmp_df["total_orders"][0])
curr_total

In [ ]:
cols_dict = {
    "curr_date": "Current Date",
    "ord_date": "Order Date",
    "seller_np": "Seller NP",
    "null_fulfilment_id": "Fulfilment ID",
    "null_net_tran_id": "Net Transaction ID",
    "null_qty": "Quantity",
    "null_itm_fulfilment_id": "Item Fulfilment ID",
    "null_del_pc": "Delivery Pincode",
    "null_created_date_time": "Created Date",
    "null_domain": "Domain",
    "null_del_cty": "Delivery City",
    "null_cans_code": "Cancellation Code",
    "null_cans_dt_time": "Cancellation Date",
    "null_ord_stats": "Order Status",
    "null_fulfil_status": "Fulfilment Status",
    "null_itm_cat": "Item Category",
    "null_cat_cons": "Category",
    "null_sell_pincode": "Seller Pincode",
    "null_prov_id": "Provider ID",
    "null_itm_id": "Item ID",
    "null_sell_np": "Null Seller NP",
    "null_net_ord_id": "Network Order ID",
    "null_sell_cty": "Seller City"
}

In [ ]:
res = []
tmp_dict = {}

for col in tmp_df.columns:
    if col.__contains__("null"):
        per = np.round((int(tmp_df[col][0])/curr_total)*100,4)
        if per > 0:
            tmp_dict = {}
            tmp_dict['title'] = cols_dict[col]
            tmp_dict['series'] = [float(per)]
            res.append(tmp_dict)

In [ ]:
res

## Totals Trend

In [ ]:
ord_trend = (
    Select(
        dq_agg_view.c.ord_date.label("ord_date"),
        func.sum(dq_agg_view.c.total_orders).label("total_orders"),
        func.sum(dq_agg_view.c.null_fulfilment_id).label("null_fulfilment_id"),
        func.sum(dq_agg_view.c.null_net_tran_id).label("null_net_tran_id"),
        func.sum(dq_agg_view.c.null_qty).label("null_qty"),
        func.sum(dq_agg_view.c.null_itm_fulfilment_id).label("null_itm_fulfilment_id"),
        func.sum(dq_agg_view.c.null_del_pc).label("null_del_pc"),
        func.sum(dq_agg_view.c.null_created_date_time).label("null_created_date_time"),
        func.sum(dq_agg_view.c.null_domain).label("null_domain"),
        func.sum(dq_agg_view.c.null_del_cty).label("null_del_cty"),
        func.sum(dq_agg_view.c.null_ord_stats).label("null_ord_stats"),
        func.sum(dq_agg_view.c.null_fulfil_status).label("null_fulfil_status"),
        func.sum(dq_agg_view.c.null_itm_cat).label("null_itm_cat"),
        func.sum(dq_agg_view.c.null_cat_cons).label("null_cat_cons"),
        func.sum(dq_agg_view.c.null_sell_pincode).label("null_sell_pincode"),
        func.sum(dq_agg_view.c.null_prov_id).label("null_prov_id"),
        func.sum(dq_agg_view.c.null_itm_id).label("null_itm_id"),
        func.sum(dq_agg_view.c.null_sell_np).label("null_sell_np"),
        func.sum(dq_agg_view.c.null_net_ord_id).label("null_net_ord_id"),
        func.sum(dq_agg_view.c.null_sell_cty).label("null_sell_cty"),
        func.sum(dq_agg_view.c.total_canceled_orders).label("total_canceled_orders"),
        func.sum(dq_agg_view.c.null_cans_code).label("null_cans_code"),
        func.sum(dq_agg_view.c.null_cans_dt_time).label("null_cans_dt_time") 
    ).group_by(dq_agg_view.c.ord_date)
    .order_by(desc(dq_agg_view.c.ord_date))
)

In [ ]:
df_trend = pd.DataFrame(run_stmt(ord_trend))

In [ ]:
df_trend.head(5)

In [ ]:
df_big = pd.DataFrame()
df_small = pd.DataFrame()

df_big[ev.cols_dict["ord_date"]] = df_trend[["ord_date"]]
df_small[ev.cols_dict["ord_date"]] = df_trend[["ord_date"]]

In [ ]:
df_big.head(5)

In [ ]:
for col in df_trend.columns:
    if col not in ["ord_date", "total_orders"]:
        try:
            mean_trend = df_trend[col].mean() 
            if mean_trend > 0:
                if mean_trend > 500:
                    df_big.loc[:,[ev.cols_dict[col]]] = df_trend[col]
                elif mean_trend < 500 and mean_trend > 1:
                    df_small.loc[:,[ev.cols_dict[col]]] = df_trend[col]
                # print(col, df_trend[col].mean())
        except:
            pass

In [ ]:
df_big.head(5)

In [ ]:
# sns.lineplot(df_big)

In [ ]:
df_big.to_json()

In [ ]:
df_small.head(5)

Jsonify

In [ ]:
df_big.info()

In [ ]:
for col in df_big.columns:
    try:
        if col != "Order Date":
            df_big.loc[:,[col]] = df_big[col].astype(int)
    except:
        pass

In [ ]:
test_dict = df_big.head(5).to_dict()

In [ ]:
test_dict.keys()

In [ ]:
res_dict = {}
output = []

In [ ]:
[val for val in test_dict["Delivery City"].values()]

In [ ]:
for key in test_dict.keys():
    if key not in ("Order Date","categories"):
        print(key)
        res_dict = {}
        res_dict["name"] = key
        res_dict["data"] = [val for val in test_dict[key].values()]
        output.append(res_dict)

Seller with Highest Missing Data 

cancel_highest_missing_pid_data

In [ ]:
stmt_curr = Select(
  dq_agg_sum.c.ord_date,
  dq_agg_sum.c.seller_np,
  func.sum(dq_agg_sum.c.total_orders).label("total_orders"),
  func.sum(dq_agg_sum.c.sum_missing_cols).label("missing_val")).where(
    dq_agg_sum.c.ord_date==max_date).group_by(
      dq_agg_sum.c.ord_date).group_by(
    dq_agg_sum.c.seller_np).order_by(desc(func.sum(dq_agg_sum.c.sum_missing_cols)))

In [ ]:
print(stmt_curr)

In [ ]:
result = run_stmt(stmt_curr, 5)

In [ ]:
df = pd.DataFrame(result)

In [ ]:
df["missing_percentage"] = df["missing_val"]/df["total_orders"]

In [ ]:
df

In [ ]:
df = df.sort_values(by="missing_percentage", ascending=False)

In [ ]:
df

In [ ]:
json_frame = {
  "id":"",
  "count":"",
  "increased":"",
  "variancePercentage":"",
  "varianceText":""
}
json_str = []
threshold = 0.05

In [ ]:
for x in df.index:
  print(x)
  json_frame = {
  "id":"",
  "count":"",
  "increased":"",
  "variancePercentage":"",
  "varianceText":""
    }
  json_frame["id"] = df.iloc[x]["seller_np"]
  json_frame["count"] = np.round(float((df.iloc[x]["missing_percentage"])),2)
  json_frame["increased"] = True if df.iloc[x]["missing_percentage"] > 0 else "False"
  json_frame["variancePercentage"] = float(threshold*100)
  json_frame["varianceText"] = "Threshold"
  print(json_frame)
  json_str.append(json_frame)


In [ ]:
json_str

### Detail Completed Table Data

In [ ]:
import Logic.base_queries as bq

In [ ]:
stmt = Select(
  dq_agg_sum.c.seller_np,
  func.sum(dq_agg_sum.c.total_orders).label("total_orders"),
  func.sum(dq_agg_sum.c.sum_missing_cols).label("sum_missing_cols")
).where(dq_agg_sum.c.ord_date==max_date).group_by(
    dq_agg_sum.c.seller_np).order_by(desc(func.sum(dq_agg_sum.c.sum_missing_cols)))

In [ ]:
max_date

In [ ]:
print(stmt)

In [ ]:
df = pd.DataFrame(run_stmt(stmt, 10))

In [ ]:
df["missing_percentage"] = df["sum_missing_cols"]/df["total_orders"]

In [ ]:
df = df.sort_values(by="missing_percentage", ascending=False)

In [ ]:
df

In [ ]:
json_str = {
    "seller_np": "",
    "null_itm_cat":0 ,
    "total_orders":0,
    "missing_percentage":0
}

json_frame = []

In [ ]:
for x in df.index:
    json_str = {
        "seller_np": "",
        "null_itm_cat":0,
        "total_orders":0,
        "missing_percentage":0
        }
    json_str["seller_np"] = df.loc[x]["seller_np"]
    json_str["null_itm_cat"] = int(df.loc[x]["sum_missing_cols"])
    json_str["total_orders"] = int(df.loc[x]["total_orders"])
    json_str["missing_percentage"] = np.round(float(df.loc[x]["missing_percentage"]),2)
    json_frame.append(json_str)

In [ ]:
json_frame

Cancelled Orders

In [ ]:
stmt = Select(
  dq_agg_sum.c.seller_np,
  func.sum(dq_agg_sum.c.total_canceled_orders).label("total_orders"),
  func.sum(dq_agg_sum.c.canc_metrices).label("sum_missing_cols")
).where(dq_agg_sum.c.ord_date==max_date).group_by(
    dq_agg_sum.c.seller_np).order_by(desc(
      func.sum(dq_agg_sum.c.total_canceled_orders)))

In [ ]:
df = pd.DataFrame(run_stmt(stmt))

In [ ]:
df.head(5)

In [ ]:
df["sum_missing_cols"].replace(0,np.nan, inplace=True)

In [ ]:
df["missing_percentage"] = df["sum_missing_cols"] / df["total_orders"]

In [ ]:
df = df.dropna()

In [ ]:
df

Trend Chart

In [ ]:
stmt = Select(dq_col_sum.c.ord_date,
       func.sum(dq_col_sum.c.null_del_cty).label("null_del_cty"),
       func.sum(dq_col_sum.c.null_itm_cat).label("null_itm_cat"),
       func.sum(dq_col_sum.c.null_cat_cons).label("null_cat_cons"),
       func.sum(dq_col_sum.c.null_cans_code).label("null_cans_code"),
       func.sum(dq_col_sum.c.null_cans_dt_time).label("null_cans_dt_time")
       ).group_by(dq_col_sum.c.ord_date)

In [ ]:
df = pd.DataFrame(run_stmt(stmt))

In [ ]:
df.head(5)

In [ ]:
# df["ord_date"]

In [ ]:
# data_ser = {
#   'name': 'col_name',
#   'date': []
# }

# cat = ['date_range']

final_json = {
  'title':'Chart Title',
  'series':[],
  'categories': []
}

In [ ]:
for col in df.columns:
    if col == 'ord_date':
        # Ensure the 'ord_date' column is treated as a string
        final_json['categories'] = df[col].astype(str).tolist()
    else:
        # Create a series dictionary
        data_ser = {
            'name': cols_dict[col],
            'data': df[col].astype(int).tolist()  # Convert column to list
        }
        
        # Add the series to the final JSON
        final_json['series'].append(data_ser)

In [ ]:
final_json['title'] = 'Columns with Highest Missing Data'

In [ ]:
final_json